In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [36]:
data_d <- '/oak/stanford/groups/mrivas/ukbb24983/cal/annotation_20200912'
in_f <- 'ukb24983_cal_cALL_v2_hg19.vep101.noLoF.tsv.gz'
out_f <- 'ukb24983_cal_cALL_v2_hg19.vep101.noLoF.Csq.tsv'


In [26]:
pvar_f <- file.path(dirname(data_d), 'pgen', 'ukb24983_cal_cALL_v2_hg19.pvar')

In [2]:
csq_rules_df <- fread('VEP_consequence_group.annotated.tsv') %>%
rename('Consequence'='#Consequence')


In [8]:
long_df <- file.path(data_d, in_f) %>%
fread(skip=94) %>%
rename('ID'='#Uploaded_variation')


In [9]:
long_df %>%
select(ID, Consequence) %>%
separate(Consequence, paste0('Consequence', 1:100), sep=',', remove=T, fill='right', extra='drop') %>%
gather(tmp, Consequence, -ID) %>%
filter(!is.na(Consequence)) %>%
select(-tmp) %>%
left_join(
    csq_rules_df %>% select(Consequence, Csq, Csq_priority),
    by='Consequence'
) %>%
unique() -> long_csq_df


In [32]:
fread(pvar_f, colClasses = c('#CHROM'='character')) %>%
rename('CHROM'='#CHROM') -> pvar


In [33]:
long_csq_df %>%
group_by(ID) %>%
arrange(Csq_priority) %>%
summarise(
    Csq = first(Csq), 
    Consequences = paste(Consequence, collapse=","), 
    .groups='drop_last'
) -> ID_Csq_map_df


In [39]:
pvar %>%
left_join(ID_Csq_map_df, by='ID') -> pvar_Csq

In [41]:
pvar_Csq %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(file.path(data_d, out_f), sep='\t', na = "NA", quote=F)


In [43]:
pvar_Csq %>%
filter(is.na(Csq))

CHROM,POS,ID,REF,ALT,Csq,Consequences
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>


In [44]:
pvar_Csq %>%
count(Csq)

Csq,n
<chr>,<int>
intron,358439
others,310295
pav,89161
pcv,11282
ptv,28321
utr,7928
